In [ ]:
# 라이브러리 설치
!pip install --upgrade pip
!pip install torch torchvision torchaudio
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!git clone https://github.com/lvyilin/pytorch-fgvc-dataset.git

# GPU 사용 확인
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
pip install scipy pillow transformers

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from  tqdm import tqdm
import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.utils import make_grid

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/stanford-cars-dataset")

100%|██████████| 1.82G/1.82G [01:01<00:00, 31.9MB/s]

Extracting files...


In [19]:
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2


In [5]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711])
])

In [28]:
from torch.utils.data import Dataset, DataLoader
train_dataset = datasets.ImageFolder('/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2/cars_train', transform=transform)
test_dataset = datasets.ImageFolder('/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2/cars_test', transform=transform)

In [197]:
os.chdir("/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2")

In [202]:
print("Sample files:", os.listdir("/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2/cars_train/cars"))

FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2/cars_train/cars'

In [164]:
import os
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import torchvision.transforms as T
from scipy.io import loadmat

# 데이터 전처리 (CLIP 요구사항에 맞는 변환)
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711])
])

# CLIP 모델 및 프로세서 로드
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# 클래스 및 라벨 로드 함수
def load_classes(meta_path):
    mat = loadmat(meta_path)
    class_names = [c[0] for c in mat['class_names'][0]]
    return class_names

def load_labels(meta_path):
    mat = loadmat(meta_path)
    annotations = mat['annotations'][0]
    labels = {}
    for ann in annotations:
        # 원본 파일 이름 (예: 'car_ims/000001.jpg')
        img_name = ann[0][0]
        # 6자리 이름을 5자리로 정규화 (예: '000001.jpg' → '00001.jpg')
        normalized_name = os.path.basename(img_name)[1:].zfill(5)
        label = int(ann[5][0, 0]) - 1  # 1-indexed -> 0-indexed
        labels[normalized_name] = label
    return labels

# 이미지 인코딩 함수
def encode_image(image_path, model, transform):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # 배치 차원 추가
    with torch.no_grad():
        image_features = model.get_image_features(image)
    return image_features / image_features.norm(p=2, dim=-1, keepdim=True)  # 정규화

# Sub-Class 텍스트 인코딩 함수
def encode_sub_classes(sub_class_list, model):
    with torch.no_grad():
        text_inputs = processor(text=sub_class_list, return_tensors="pt", padding=True)
        text_features = model.get_text_features(**text_inputs)
    return text_features / text_features.norm(p=2, dim=-1, keepdim=True)  # 정규화

# 유사도 계산 함수
def calculate_similarity(image_features, text_features):
    return torch.matmul(image_features, text_features.T).squeeze(0).tolist()

# 클래스 점수 계산 및 분류
def classify_image(image_path, sub_classes, model, transform):
    image_features = encode_image(image_path, model, transform)
    class_scores = {}
    sub_class_scores_dict = {}

    for class_name, sub_class_list in sub_classes.items():
        text_features = encode_sub_classes(sub_class_list, model)
        sub_class_scores = calculate_similarity(image_features, text_features)
        sub_class_scores_dict[class_name] = sub_class_scores
        class_score = sum(sub_class_scores)  # Sub-Class 점수 합산
        class_scores[class_name] = class_score

    # 가장 높은 점수의 클래스 선택
    predicted_class = max(class_scores, key=class_scores.get)
    return predicted_class, class_scores, sub_class_scores_dict

# 데이터셋 경로 정보
dataset_path = "/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2"
image_dir = os.path.join(dataset_path, "cars_train", "cars_train")  # 실제 이미지 디렉토리
meta_file = os.path.join(dataset_path, 'cars_annos.mat')

# 클래스 및 라벨 로드
class_names = load_classes(meta_file)
labels = load_labels(meta_file)

# 테스트 이미지
image_name = "01000.jpg"  # 수정된 5자리 파일 이름
image_path = os.path.join(image_dir, image_name)

# 실제 라벨
true_label_index = labels[image_name]
true_label_name = class_names[true_label_index]

# 이미지 분류
predicted_class, class_scores, sub_class_scores_dict = classify_image(image_path, sub_classes, model, transform)

# 결과 출력
print(f"True Label: {true_label_name}")
print(f"Predicted Class: {predicted_class}\n")

print("Class Scores:")
for class_name, score in class_scores.items():
    print(f"- {class_name}: {score:.4f}")

print("\nSub-Class Scores:")
for class_name, sub_class_scores in sub_class_scores_dict.items():
    print(f"\n{class_name}:")
    for sub_class, score in zip(sub_classes[class_name], sub_class_scores):
        print(f"- {sub_class}: {score:.4f}")

True Label: Audi A5 Coupe 2012
Predicted Class: Dodge Sprinter Cargo Van 2009

Class Scores:
- AM General Hummer SUV 2000: 1.2348
- Acura RL Sedan 2012: 1.0515
- Acura TL Sedan 2012: 1.0902
- Acura TL Type-S 2008: 1.0395
- Acura TSX Sedan 2012: 1.0763
- Acura Integra Type R 2001: 1.0799
- Acura ZDX Hatchback 2012: 1.0990
- Aston Martin V8 Vantage Convertible 2012: 1.0786
- Aston Martin V8 Vantage Coupe 2012: 1.0567
- Aston Martin Virage Convertible 2012: 1.0770
- Aston Martin Virage Coupe 2012: 1.0059
- Audi RS 4 Convertible 2008: 1.0826
- Audi A5 Coupe 2012: 0.9815
- Audi TTS Coupe 2012: 1.0956
- Audi R8 Coupe 2012: 0.9851
- Audi 100 Sedan 1994: 1.0651
- Audi 100 Wagon 1994: 1.1546
- Audi TT Hatchback 2011: 1.1026
- Audi S6 Sedan 2011: 1.0820
- Audi S5 Convertible 2012: 1.0788
- Audi S5 Coupe 2012: 1.0638
- Audi S4 Sedan 2012: 1.0627
- Audi S4 Sedan 2007: 1.0651
- Audi TT RS Coupe 2012: 1.0732
- BMW ActiveHybrid 5 Sedan 2012: 1.1746
- BMW 1 Series Convertible 2012: 1.1483
- BMW 1 Seri

In [175]:
print("CUDA Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA Available: True
Current Device: 0
Device Name: Tesla T4


In [210]:
import os
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
import torchvision.transforms as T
from tqdm import tqdm
from scipy.io import loadmat
import pandas as pd

# 메타파일 경로
meta_file = "/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2/cars_annos.mat"
image_dir = "/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2/cars_test/cars_test"

# 메타데이터 로드
mat = loadmat(meta_file)
annotations = mat['annotations'][0]
class_names = [c[0] for c in mat['class_names'][0]]

# Test 데이터 필터링 및 재인덱싱
test_annotations = [
    {
        "Image Path": ann[0][0],
        "Normalized Name": os.path.basename(ann[0][0])[1:].zfill(5),
        "Label Index": int(ann[5][0, 0]) - 1,
        "Is Test": int(ann[6][0, 0])
    }
    for ann in annotations if int(ann[6][0, 0]) == 1
]

# 정렬 및 새로운 파일 이름 생성
test_annotations = sorted(test_annotations, key=lambda x: int(x["Normalized Name"].replace(".jpg", "")))
for idx, ann in enumerate(test_annotations, start=1):
    ann["Reindexed Name"] = str(idx).zfill(5) + ".jpg"

# CLIP 모델 및 프로세서 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device).eval()
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# 데이터 변환 설정
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711])
])

# 클래스 텍스트 특징 미리 계산
text_inputs = processor(text=class_names, return_tensors="pt", padding=True).to(device)
with torch.no_grad():
    text_features = model.get_text_features(**text_inputs)
    text_features = text_features / text_features.norm(p=2, dim=-1, keepdim=True)

# 평가 시작
correct_predictions = 0
total_images = len(test_annotations)

print("Evaluating CLIP Zero-Shot Performance on Reindexed Test Data...")
for ann in tqdm(test_annotations, total=total_images):
    reindexed_name = ann["Reindexed Name"]
    label_name = class_names[ann["Label Index"]]
    image_path = os.path.join(image_dir, reindexed_name)

    # 이미지 특징 계산
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"File not found: {image_path}")
        continue  # 누락된 파일 건너뛰기

    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.get_image_features(image)
        image_features = image_features / image_features.norm(p=2, dim=-1, keepdim=True)

        # 유사도 계산
        similarities = torch.matmul(image_features, text_features.T)
        predicted_index = similarities.argmax(dim=-1).item()
        predicted_label_name = class_names[predicted_index]

    # 정확도 계산
    if predicted_label_name == label_name:
        correct_predictions += 1

# 최종 정확도 계산
accuracy = correct_predictions / total_images * 100
print(f"CLIP Zero-Shot Accuracy on Reindexed Test Data: {accuracy:.2f}%")

Evaluating CLIP Zero-Shot Performance on Reindexed Test Data...


100%|██████████| 8041/8041 [02:02<00:00, 65.80it/s]

CLIP Zero-Shot Accuracy on Reindexed Test Data: 0.57%


###sub_classes

In [162]:
from scipy.io import loadmat

# 데이터셋 메타파일 경로
meta_file = "/root/.cache/kagglehub/datasets/jessicali9530/stanford-cars-dataset/versions/2/cars_annos.mat"

# 클래스 이름 로드 함수
def load_classes(meta_path):
    mat = loadmat(meta_path)  # .mat 파일 로드
    class_names = [c[0] for c in mat['class_names'][0]]  # 클래스 이름 추출
    return class_names

# 클래스 이름 로드
class_names = load_classes(meta_file)

# 클래스 이름 출력
print("Stanford Cars Dataset Classes:")
for i, class_name in enumerate(class_names, 1):
    print(f"{i}: {class_name}")

Stanford Cars Dataset Classes:
1: AM General Hummer SUV 2000
2: Acura RL Sedan 2012
3: Acura TL Sedan 2012
4: Acura TL Type-S 2008
5: Acura TSX Sedan 2012
6: Acura Integra Type R 2001
7: Acura ZDX Hatchback 2012
8: Aston Martin V8 Vantage Convertible 2012
9: Aston Martin V8 Vantage Coupe 2012
10: Aston Martin Virage Convertible 2012
11: Aston Martin Virage Coupe 2012
12: Audi RS 4 Convertible 2008
13: Audi A5 Coupe 2012
14: Audi TTS Coupe 2012
15: Audi R8 Coupe 2012
16: Audi V8 Sedan 1994
17: Audi 100 Sedan 1994
18: Audi 100 Wagon 1994
19: Audi TT Hatchback 2011
20: Audi S6 Sedan 2011
21: Audi S5 Convertible 2012
22: Audi S5 Coupe 2012
23: Audi S4 Sedan 2012
24: Audi S4 Sedan 2007
25: Audi TT RS Coupe 2012
26: BMW ActiveHybrid 5 Sedan 2012
27: BMW 1 Series Convertible 2012
28: BMW 1 Series Coupe 2012
29: BMW 3 Series Sedan 2012
30: BMW 3 Series Wagon 2012
31: BMW 6 Series Convertible 2007
32: BMW X5 SUV 2007
33: BMW X6 SUV 2012
34: BMW M3 Coupe 2012
35: BMW M5 Sedan 2010
36: BMW M6 Con

In [163]:
# Sub-Class 정의
sub_classes = {
    "AM General Hummer SUV 2000": [
        "Robust and durable exterior design",
        "Exceptional off-road performance",
        "Large and spacious SUV for rough terrains",
        "High ground clearance",
        "Derived from military vehicles"
    ],
    "Acura RL Sedan 2012": [
        "Elegant and refined exterior design",
        "Comfortable high-performance sedan",
        "Suitable for city and highway driving",
        "Midsize luxury sedan",
        "A reliable model from Acura"
    ],
    "Acura TL Sedan 2012": [
        "Stylish and aerodynamic design",
        "Smooth and powerful performance",
        "Ideal for daily commuting and long trips",
        "Compact executive sedan",
        "Modern technology and luxury features"
    ],
    "Acura TL Type-S 2008": [
        "Sporty and aggressive design",
        "High-performance V6 engine",
        "Focused on driving enthusiasts",
        "Midsize sedan with sporty appeal",
        "A legacy of Acura's performance line"
    ],
    "Acura TSX Sedan 2012": [
        "Sophisticated and sleek exterior",
        "Balanced and reliable performance",
        "Perfect for daily city commuting",
        "Compact executive sedan",
        "Known for fuel efficiency and reliability"
    ],
    "Acura Integra Type R 2001": [
        "Aggressive and sporty exterior",
        "High-revving naturally aspirated engine",
        "Designed for track enthusiasts",
        "Compact performance coupe",
        "An iconic model in sports car history"
    ],
    "Acura ZDX Hatchback 2012": [
        "Modern and unique crossover design",
        "Sporty and agile performance",
        "Ideal for urban and suburban driving",
        "Compact luxury crossover",
        "Equipped with advanced Acura technologies"
    ],
    "Aston Martin V8 Vantage Convertible 2012": [
        "Elegant and sleek convertible design",
        "Powerful V8 engine performance",
        "Built for open-road luxury driving",
        "Compact luxury convertible",
        "A symbol of British automotive craftsmanship"
    ],
    "Aston Martin V8 Vantage Coupe 2012": [
        "Stylish and muscular coupe design",
        "High-performance V8 engine",
        "Ideal for spirited and luxurious drives",
        "Compact luxury sports coupe",
        "Renowned for its handcrafted quality"
    ],
    "Aston Martin Virage Convertible 2012": [
        "Timeless and sophisticated design",
        "Refined V12 engine performance",
        "Perfect for open-top grand touring",
        "Large luxury convertible",
        "Blends performance with luxury craftsmanship"
    ],
    "Aston Martin Virage Coupe 2012": [
        "Iconic and graceful coupe design",
        "Exceptional V12 engine performance",
        "Built for grand touring experiences",
        "Large luxury grand tourer coupe",
        "A showcase of British engineering excellence"
    ],
    "Audi RS 4 Convertible 2008": [
        "Aggressive and sporty convertible design",
        "High-performance V8 engine",
        "Engineered for high-speed open-top driving",
        "Compact luxury sports convertible",
        "Renowned for Audi's Quattro technology"
    ],
    "Audi A5 Coupe 2012": [
        "Modern and aerodynamic coupe design",
        "Balanced and smooth performance",
        "Ideal for urban and highway drives",
        "Compact luxury coupe",
        "Showcases Audi's design language"
    ],
    "Audi TTS Coupe 2012": [
        "Compact and sporty coupe design",
        "Turbocharged engine for high performance",
        "Engineered for spirited driving",
        "Compact sports coupe",
        "Known for agility and Quattro AWD system"
    ],
    "Audi R8 Coupe 2012": [
        "Exotic and aggressive supercar design",
        "V10 engine with thrilling performance",
        "Perfect for high-speed track driving",
        "Luxury sports coupe",
        "Showcases Audi's flagship engineering"
    ],
    "Audi 100 Sedan 1994": [
        "Classic and simple sedan design",
        "Reliable performance with smooth handling",
        "Ideal for daily urban commuting",
        "Midsize executive sedan",
        "Represents Audi's engineering evolution in the '90s"
    ],
    "Audi 100 Wagon 1994": [
        "Spacious and practical wagon design",
        "Balanced performance with comfort",
        "Perfect for family and cargo transport",
        "Large station wagon",
        "A reliable model from Audi's '90s lineup"
    ],
    "Audi TT Hatchback 2011": [
        "Sporty and compact hatchback design",
        "Turbocharged engine for agile performance",
        "Ideal for urban and suburban driving",
        "Compact luxury hatchback",
        "Represents Audi's innovative hatchback series"
    ],
    "Audi S6 Sedan 2011": [
        "Elegant and sporty sedan design",
        "High-performance engine with smooth acceleration",
        "Suitable for city driving and highway cruising",
        "Midsize luxury sedan",
        "Showcases Audi's S-line engineering"
    ],
    "Audi S5 Convertible 2012": [
        "Stylish and dynamic convertible design",
        "High-powered engine for open-top performance",
        "Engineered for spirited luxury driving",
        "Compact luxury convertible",
        "Represents Audi's commitment to premium quality"
    ],
    "Audi S5 Coupe 2012": [
        "Aggressive and sleek coupe design",
        "Performance-tuned engine for smooth drives",
        "Ideal for high-speed highway cruising",
        "Compact sports coupe",
        "A refined model from Audi's S-line series"
    ],
    "Audi S4 Sedan 2012": [
        "Modern and understated sporty design",
        "Turbocharged engine for thrilling performance",
        "Designed for executive-level commuting",
        "Compact executive sedan",
        "Blends Audi's luxury with sportiness"
    ],
    "Audi S4 Sedan 2007": [
        "Classic and sporty sedan design",
        "High-performance V8 engine",
        "Perfect for spirited yet comfortable driving",
        "Midsize sports sedan",
        "Represents Audi's earlier S-line refinement"
    ],
    "Audi TT RS Coupe 2012": [
        "Compact and aggressive coupe design",
        "Turbocharged engine with precise handling",
        "Engineered for track and spirited drives",
        "Small luxury sports coupe",
        "Audi's benchmark for compact performance cars"
    ],
    "BMW ActiveHybrid 5 Sedan 2012": [
        "Modern and eco-friendly sedan design",
        "Hybrid engine offering efficiency and power",
        "Perfect for urban and long-distance driving",
        "Midsize luxury hybrid sedan",
        "Combines BMW's engineering with green technology"
    ],
    "BMW 1 Series Convertible 2012": [
        "Compact and stylish convertible design",
        "Balanced engine for smooth performance",
        "Ideal for open-road driving",
        "Small luxury convertible",
        "Represents BMW's entry-level luxury line"
    ],
    "BMW 1 Series Coupe 2012": [
        "Sporty and refined compact coupe design",
        "High-performance engine for dynamic driving",
        "Built for urban and spirited highway driving",
        "Compact executive coupe",
        "Combines agility with BMW's luxury standards"
    ],
    "BMW 3 Series Sedan 2012": [
        "Iconic and versatile sedan design",
        "Smooth and powerful engine performance",
        "Suitable for commuting and road trips",
        "Midsize executive sedan",
        "A cornerstone of BMW's lineup"
    ],
    "BMW 3 Series Wagon 2012": [
        "Spacious and functional wagon design",
        "Balanced performance and handling",
        "Perfect for family trips and cargo needs",
        "Large luxury station wagon",
        "Combines practicality with BMW's luxury features"
    ],
    "BMW 6 Series Convertible 2007": [
        "Elegant and premium convertible design",
        "Powerful engine with smooth acceleration",
        "Engineered for luxurious open-top touring",
        "Large luxury convertible",
        "Showcases BMW's craftsmanship and elegance"
    ],
    "BMW X5 SUV 2007": [
        "Strong and robust SUV design",
        "High-performance engine with off-road capability",
        "Ideal for family and adventurous drives",
        "Large luxury SUV",
        "A hallmark of BMW's SUV engineering"
    ],
    "BMW X6 SUV 2012": [
        "Sporty and futuristic SUV design",
        "High-powered engine for smooth performance",
        "Perfect for urban and suburban driving",
        "Midsize luxury SUV",
        "Represents BMW's innovation in SUV design"
    ],
    "BMW M3 Coupe 2012": [
        "Aggressive and iconic coupe design",
        "High-revving performance-tuned engine",
        "Built for track and spirited highway driving",
        "Compact luxury sports coupe",
        "A symbol of BMW's M-performance line"
    ],
    "BMW M5 Sedan 2010": [
        "Refined and sporty sedan design",
        "Powerful engine for thrilling performance",
        "Ideal for high-speed executive commuting",
        "Midsize luxury sports sedan",
        "A flagship model in BMW's M lineup"
    ],
    "BMW M6 Convertible 2010": [
        "Elegant and sporty convertible design",
        "V10 engine delivering high-performance",
        "Designed for luxurious open-top driving",
        "Large luxury convertible",
        "A hallmark of BMW's M-series engineering"
    ],
    "BMW X3 SUV 2012": [
        "Modern and compact SUV design",
        "Balanced engine performance and agility",
        "Ideal for urban and off-road adventures",
        "Compact luxury SUV",
        "A versatile model in BMW's X-series"
    ],
    "BMW Z4 Convertible 2012": [
        "Stylish and sleek convertible design",
        "Turbocharged engine for spirited driving",
        "Perfect for open-road cruising",
        "Small luxury convertible",
        "Represents BMW's craftsmanship in roadsters"
    ],
    "Bentley Continental Supersports Conv. Convertible 2012": [
        "Luxurious and aerodynamic convertible design",
        "W12 engine with incredible performance",
        "Engineered for high-speed grand touring",
        "Large luxury convertible",
        "A flagship model in Bentley's lineup"
    ],
    "Bentley Arnage Sedan 2009": [
        "Timeless and elegant sedan design",
        "V8 engine with smooth and powerful performance",
        "Built for ultimate luxury commuting",
        "Large luxury sedan",
        "Represents Bentley's heritage in craftsmanship"
    ],
    "Bentley Mulsanne Sedan 2011": [
        "Prestigious and bold exterior design",
        "V8 engine offering refined performance",
        "Designed for executive-level luxury",
        "Full-size luxury sedan",
        "A symbol of Bentley's commitment to excellence"
    ],
    "Bentley Continental GT Coupe 2012": [
        "Elegant and muscular coupe design",
        "Twin-turbocharged W12 engine",
        "Ideal for high-speed grand touring",
        "Large luxury coupe",
        "An iconic model in Bentley's lineup"
    ],
    "Bentley Continental GT Coupe 2007": [
        "Classic and graceful coupe design",
        "Powerful and smooth engine performance",
        "Built for luxurious long-distance driving",
        "Large luxury grand tourer coupe",
        "A timeless representation of Bentley's excellence"
    ],
    "Bentley Continental Flying Spur Sedan 2007": [
        "Luxurious and spacious sedan design",
        "W12 engine for seamless performance",
        "Perfect for executive-level commuting",
        "Full-size luxury sedan",
        "A cornerstone of Bentley's sedan lineup"
    ],
    "Bugatti Veyron 16.4 Convertible 2009": [
        "Exotic and aerodynamic convertible design",
        "Quad-turbocharged W16 engine with extreme power",
        "Engineered for high-speed luxury driving",
        "Compact hypercar convertible",
        "Represents Bugatti's engineering prowess"
    ],
    "Bugatti Veyron 16.4 Coupe 2009": [
        "Futuristic and aggressive coupe design",
        "Record-breaking performance with W16 engine",
        "Designed for unmatched speed and luxury",
        "Compact hypercar coupe",
        "A benchmark in automotive engineering history"
    ],
    "Buick Regal GS 2012": [
        "Modern and sophisticated sedan design",
        "Turbocharged engine for smooth performance",
        "Perfect for city and highway driving",
        "Midsize luxury sedan",
        "Combines Buick's reliability with modern design"
    ],
    "Buick Rainier SUV 2007": [
        "Robust and practical SUV design",
        "V8 engine with balanced performance",
        "Ideal for family and cargo transport",
        "Midsize luxury SUV",
        "A dependable model in Buick's SUV lineup"
    ],
    "Buick Verano Sedan 2012": [
        "Stylish and compact sedan design",
        "Efficient engine with smooth handling",
        "Engineered for urban commuting",
        "Compact luxury sedan",
        "Represents Buick's modern design language"
    ],
    "Buick Enclave SUV 2012": [
        "Spacious and elegant SUV design",
        "V6 engine delivering smooth performance",
        "Perfect for family and long trips",
        "Large luxury SUV",
        "Combines practicality with Buick's luxury features"
    ],
    "Cadillac CTS-V Sedan 2012": [
        "Aggressive and sporty sedan design",
        "Supercharged V8 engine for thrilling performance",
        "Built for high-speed luxury commuting",
        "Midsize luxury sports sedan",
        "A flagship model in Cadillac's V-series lineup"
    ],
    "Cadillac SRX SUV 2012": [
        "Modern and aerodynamic SUV design",
        "Balanced engine performance for smooth rides",
        "Ideal for family trips and daily commuting",
        "Midsize luxury SUV",
        "A key model in Cadillac's SUV offerings"
    ],
    "Cadillac Escalade EXT Crew Cab 2007": [
        "Bold and imposing exterior design",
        "V8 engine offering seamless performance",
        "Designed for luxury and utility",
        "Full-size luxury crew cab",
        "A unique blend of SUV and truck in Cadillac's lineup"
    ],
    "Chevrolet Silverado 1500 Hybrid Crew Cab 2012": [
        "Robust and eco-friendly truck design",
        "Hybrid engine offering efficiency and power",
        "Engineered for cargo and towing tasks",
        "Full-size hybrid crew cab truck",
        "Represents Chevrolet's innovation in trucks"
    ],
    "Chevrolet Corvette Convertible 2012": [
        "Sleek and iconic convertible design",
        "V8 engine for high-performance driving",
        "Ideal for open-road adventures",
        "Compact luxury sports convertible",
        "A legendary name in American sports cars"
    ],
    "Chevrolet Corvette ZR1 2012": [
        "Aggressive and aerodynamic coupe design",
        "Supercharged V8 engine for extreme performance",
        "Designed for track and high-speed driving",
        "Compact luxury sports coupe",
        "Represents the pinnacle of Corvette engineering"
    ],
    "Chevrolet Corvette Ron Fellows Edition Z06 2007": [
        "Bold and unique limited-edition design",
        "High-performance V8 engine",
        "Engineered for track enthusiasts",
        "Compact sports coupe",
        "Commemorates racing legend Ron Fellows"
    ],
    "Chevrolet Traverse SUV 2012": [
        "Spacious and modern SUV design",
        "V6 engine offering balanced performance",
        "Ideal for family and cargo transportation",
        "Large luxury SUV",
        "A dependable model in Chevrolet's lineup"
    ],
    "Chevrolet Camaro Convertible 2012": [
        "Muscular and stylish convertible design",
        "V8 engine with thrilling performance",
        "Engineered for open-top driving excitement",
        "Compact luxury sports convertible",
        "A symbol of American muscle cars"
    ],
    "Chevrolet HHR SS 2010": [
        "Retro-inspired compact design",
        "Turbocharged engine for sporty performance",
        "Perfect for urban and suburban driving",
        "Compact performance wagon",
        "Represents Chevrolet's modern take on retro design"
    ],
    "Chevrolet Impala Sedan 2007": [
        "Classic and timeless sedan design",
        "Reliable V6 engine with smooth performance",
        "Ideal for commuting and family use",
        "Midsize luxury sedan",
        "A long-standing name in Chevrolet's history"
    ],
    "Chevrolet Tahoe Hybrid SUV 2012": [
        "Robust and eco-friendly SUV design",
        "Hybrid engine offering efficiency and power",
        "Perfect for family trips and off-road adventures",
        "Large luxury hybrid SUV",
        "Combines utility with Chevrolet's green technology"
    ],
    "Chevrolet Sonic Sedan 2012": [
        "Compact and modern sedan design",
        "Efficient engine for daily commuting",
        "Engineered for urban driving",
        "Small luxury sedan",
        "Represents Chevrolet's focus on compact cars"
    ],
    "Chevrolet Express Cargo Van 2007": [
        "Practical and functional van design",
        "V8 engine with strong towing capabilities",
        "Ideal for cargo and commercial use",
        "Large utility van",
        "A reliable model in Chevrolet's utility lineup"
    ],
    "Chevrolet Avalanche Crew Cab 2012": [
        "Bold and versatile truck design",
        "V8 engine with smooth performance",
        "Designed for towing and off-road tasks",
        "Full-size crew cab truck",
        "Combines SUV comfort with truck utility"
    ],
    "Chevrolet Cobalt SS 2010": [
        "Sporty and compact sedan design",
        "Turbocharged engine for spirited driving",
        "Perfect for urban and highway use",
        "Small performance sedan",
        "Represents Chevrolet's commitment to compact performance"
    ],
    "Chevrolet Malibu Hybrid Sedan 2010": [
        "Eco-friendly and modern sedan design",
        "Hybrid engine delivering efficiency and reliability",
        "Ideal for daily commuting and long trips",
        "Midsize hybrid sedan",
        "Represents Chevrolet's green initiatives"
    ],
    "Chevrolet TrailBlazer SS 2009": [
        "Strong and aggressive SUV design",
        "Performance-tuned V8 engine",
        "Engineered for sporty and off-road driving",
        "Midsize luxury SUV",
        "A sporty addition to Chevrolet's SUV lineup"
    ],
    "Chevrolet Silverado 2500HD Regular Cab 2012": [
        "Durable and functional truck design",
        "Heavy-duty engine for towing and hauling",
        "Designed for work and utility",
        "Full-size heavy-duty regular cab truck",
        "A cornerstone of Chevrolet's truck offerings"
    ],
    "Chevrolet Silverado 1500 Classic Extended Cab 2007": [
        "Classic and robust truck design",
        "Balanced engine performance for utility tasks",
        "Ideal for towing and family transport",
        "Full-size extended cab truck",
        "Represents Chevrolet's dependable truck legacy"
    ],
    "Chevrolet Express Van 2007": [
        "Spacious and versatile van design",
        "Strong engine with reliable performance",
        "Perfect for commercial and family use",
        "Large utility van",
        "A dependable model in Chevrolet's van lineup"
    ],
    "Chevrolet Monte Carlo Coupe 2007": [
        "Sporty and iconic coupe design",
        "V6 engine delivering smooth performance",
        "Engineered for urban and highway driving",
        "Midsize luxury coupe",
        "A classic nameplate in Chevrolet's history"
    ],
    "Chevrolet Malibu Sedan 2007": [
        "Timeless and elegant sedan design",
        "Reliable engine with balanced performance",
        "Designed for commuting and family trips",
        "Midsize luxury sedan",
        "A hallmark of Chevrolet's sedan lineup"
    ],
    "Chevrolet Silverado 1500 Extended Cab 2012": [
        "Versatile and durable truck design",
        "V8 engine offering strong towing capabilities",
        "Perfect for work and family use",
        "Full-size extended cab truck",
        "A key model in Chevrolet's truck offerings"
    ],
    "Chevrolet Silverado 1500 Regular Cab 2012": [
        "Classic and functional truck design",
        "V8 engine with dependable performance",
        "Ideal for utility and everyday use",
        "Full-size regular cab truck",
        "Represents Chevrolet's reliable truck legacy"
    ],
    "Chrysler Aspen SUV 2009": [
        "Bold and rugged SUV design",
        "V8 engine with smooth performance",
        "Perfect for family and cargo transport",
        "Midsize luxury SUV",
        "Represents Chrysler's foray into SUVs"
    ],
    "Chrysler Sebring Convertible 2010": [
        "Elegant and practical convertible design",
        "Balanced engine for reliable performance",
        "Engineered for open-top urban and highway driving",
        "Compact luxury convertible",
        "A versatile model in Chrysler's lineup"
    ],
    "Chrysler Town and Country Minivan 2012": [
        "Spacious and modern minivan design",
        "V6 engine delivering smooth performance",
        "Ideal for family trips and urban driving",
        "Large luxury minivan",
        "Represents Chrysler's commitment to family vehicles"
    ],
    "Chrysler 300 SRT-8 2010": [
        "Bold and aggressive sedan design",
        "High-performance V8 engine",
        "Built for high-speed and luxury commuting",
        "Full-size luxury sports sedan",
        "A flagship model in Chrysler's lineup"
    ],
    "Chrysler Crossfire Convertible 2008": [
        "Stylish and compact convertible design",
        "Sporty engine with balanced performance",
        "Ideal for open-top urban driving",
        "Compact luxury convertible",
        "Represents Chrysler's unique design philosophy"
    ],
    "Chrysler PT Cruiser Convertible 2008": [
        "Retro and unique convertible design",
        "Balanced engine for daily performance",
        "Perfect for urban and suburban driving",
        "Compact luxury convertible",
        "Represents Chrysler's creative design approach"
    ],
    "Daewoo Nubira Wagon 2002": [
        "Simple and practical wagon design",
        "Efficient engine for daily commuting",
        "Engineered for family and cargo transport",
        "Compact wagon",
        "A dependable model in Daewoo's lineup"
    ],
    "Dodge Caliber Wagon 2012": [
        "Rugged and modern wagon design",
        "Efficient engine with balanced performance",
        "Ideal for urban and suburban commuting",
        "Compact utility wagon",
        "A versatile model in Dodge's lineup"
    ],
    "Dodge Caliber Wagon 2007": [
        "Bold and practical wagon design",
        "Reliable engine for everyday driving",
        "Perfect for family and cargo needs",
        "Compact wagon",
        "Represents Dodge's practicality in design"
    ],
    "Dodge Caravan Minivan 1997": [
        "Classic and spacious minivan design",
        "Balanced engine performance for family use",
        "Ideal for long-distance trips and cargo transport",
        "Large family minivan",
        "A pioneer in Dodge's minivan lineup"
    ],
    "Dodge Ram Pickup 3500 Crew Cab 2010": [
        "Bold and robust truck design",
        "Heavy-duty engine for towing and hauling",
        "Built for work and utility tasks",
        "Full-size heavy-duty crew cab truck",
        "A key model in Dodge's truck offerings"
    ],
    "Dodge Ram Pickup 3500 Quad Cab 2009": [
        "Durable and functional truck design",
        "Powerful engine for heavy-duty performance",
        "Engineered for utility and cargo needs",
        "Full-size heavy-duty quad cab truck",
        "Represents Dodge's dedication to trucks"
    ],
    "Dodge Sprinter Cargo Van 2009": [
        "Spacious and practical van design",
        "Strong engine with reliable performance",
        "Ideal for commercial and cargo use",
        "Large utility van",
        "A dependable model in Dodge's van lineup"
    ],
    "Dodge Journey SUV 2012": [
        "Modern and versatile SUV design",
        "Efficient engine for smooth performance",
        "Perfect for family trips and daily commuting",
        "Midsize luxury SUV",
        "Represents Dodge's commitment to family vehicles"
    ],
    "Dodge Dakota Crew Cab 2010": [
        "Compact and robust truck design",
        "Balanced engine performance for utility tasks",
        "Engineered for towing and hauling",
        "Midsize crew cab truck",
        "A reliable model in Dodge's lineup"
    ],
    "Dodge Dakota Club Cab 2007": [
        "Classic and durable truck design",
        "Efficient engine for work and utility",
        "Perfect for daily driving and cargo needs",
        "Compact club cab truck",
        "Represents Dodge's practical truck offerings"
    ],
    "Dodge Magnum Wagon 2008": [
        "Bold and muscular wagon design",
        "V6 engine with smooth performance",
        "Engineered for family and cargo transport",
        "Large luxury wagon",
        "A unique model in Dodge's lineup"
    ],
    "Dodge Challenger SRT8 2011": [
        "Aggressive and iconic muscle car design",
        "High-performance V8 engine",
        "Built for track and highway driving",
        "Midsize luxury sports coupe",
        "Represents the legacy of Dodge muscle cars"
    ],
    "Dodge Durango SUV 2012": [
        "Strong and modern SUV design",
        "V6 engine delivering balanced performance",
        "Perfect for family trips and off-road adventures",
        "Large luxury SUV",
        "A cornerstone of Dodge's SUV lineup"
    ],
    "Dodge Durango SUV 2007": [
        "Classic and robust SUV design",
        "Reliable engine for everyday use",
        "Ideal for family transport and light off-roading",
        "Large family SUV",
        "Represents Dodge's dependable SUV offerings"
    ],
    "Dodge Charger Sedan 2012": [
        "Bold and sleek sedan design",
        "V8 engine with high-performance capability",
        "Ideal for high-speed highway driving",
        "Full-size luxury sports sedan",
        "Represents Dodge's legacy of powerful sedans"
    ],
    "Dodge Charger SRT-8 2009": [
        "Aggressive and sporty sedan design",
        "Supercharged V8 engine for extreme performance",
        "Built for track enthusiasts and spirited driving",
        "Midsize luxury sports sedan",
        "A performance model in Dodge's lineup"
    ],
    "Eagle Talon Hatchback 1998": [
        "Compact and aerodynamic hatchback design",
        "Turbocharged engine for sporty performance",
        "Perfect for urban and suburban driving",
        "Small performance hatchback",
        "An iconic model in the Eagle lineup"
    ],
    "FIAT 500 Abarth 2012": [
        "Compact and stylish hatchback design",
        "Turbocharged engine for zippy performance",
        "Engineered for urban and city driving",
        "Small luxury hatchback",
        "A sporty iteration of the iconic Fiat 500"
    ],
    "FIAT 500 Convertible 2012": [
        "Charming and compact convertible design",
        "Efficient engine for smooth performance",
        "Ideal for open-top urban driving",
        "Small convertible hatchback",
        "Represents Fiat's blend of style and function"
    ],
    "Ferrari FF Coupe 2012": [
        "Elegant and futuristic coupe design",
        "V12 engine for exceptional performance",
        "Designed for high-speed grand touring",
        "Large luxury sports coupe",
        "A symbol of Ferrari's innovation and luxury"
    ],
    "Ferrari California Convertible 2012": [
        "Luxurious and sleek convertible design",
        "V8 engine delivering high-speed thrills",
        "Built for open-top grand touring",
        "Large luxury convertible",
        "Represents Ferrari's craftsmanship and elegance"
    ],
    "Ferrari 458 Italia Convertible 2012": [
        "Aerodynamic and exotic convertible design",
        "V8 engine with track-level performance",
        "Engineered for high-speed open-top driving",
        "Compact hypercar convertible",
        "A standout model in Ferrari's lineup"
    ],
    "Ferrari 458 Italia Coupe 2012": [
        "Aggressive and refined coupe design",
        "V8 engine delivering exhilarating performance",
        "Designed for track and highway excellence",
        "Compact hypercar coupe",
        "Represents Ferrari's commitment to performance"
    ],
    "Fisker Karma Sedan 2012": [
        "Futuristic and eco-friendly sedan design",
        "Hybrid engine delivering efficient performance",
        "Perfect for urban commuting and long trips",
        "Midsize luxury hybrid sedan",
        "Represents Fisker's innovation in green technology"
    ],
    "Ford F-450 Super Duty Crew Cab 2012": [
        "Heavy-duty and robust truck design",
        "Powerful engine for towing and hauling",
        "Engineered for utility and industrial tasks",
        "Full-size heavy-duty crew cab truck",
        "A benchmark in Ford's Super Duty lineup"
    ],
    "Ford Mustang Convertible 2007": [
        "Iconic and sporty convertible design",
        "V8 engine delivering thrilling performance",
        "Built for open-top driving fun",
        "Compact luxury convertible",
        "A classic in American muscle cars"
    ],
    "Ford Freestar Minivan 2007": [
        "Spacious and practical minivan design",
        "V6 engine with reliable performance",
        "Ideal for family trips and daily transport",
        "Large family minivan",
        "Represents Ford's dedication to family vehicles"
    ],
    "Ford Expedition EL SUV 2009": [
        "Bold and spacious SUV design",
        "V8 engine with balanced performance",
        "Perfect for family and off-road adventures",
        "Full-size luxury SUV",
        "A cornerstone in Ford's SUV lineup"
    ],
    "Ford Edge SUV 2012": [
        "Modern and aerodynamic SUV design",
        "Balanced engine delivering smooth performance",
        "Ideal for urban and suburban driving",
        "Midsize luxury SUV",
        "Represents Ford's innovation in SUVs"
    ],
    "Ford Ranger SuperCab 2011": [
        "Compact and functional truck design",
        "Efficient engine with reliable performance",
        "Engineered for light-duty tasks and transport",
        "Compact utility truck",
        "A versatile model in Ford's lineup"
    ],
    "Ford GT Coupe 2006": [
        "Exotic and aggressive supercar design",
        "Supercharged V8 engine for record-breaking performance",
        "Built for track and high-speed excellence",
        "Compact hypercar coupe",
        "Represents Ford's engineering masterpiece"
    ],
    "Ford F-150 Regular Cab 2012": [
        "Classic and durable truck design",
        "Powerful engine for towing and hauling",
        "Ideal for work and everyday use",
        "Full-size regular cab truck",
        "A reliable cornerstone in Ford's lineup"
    ],
    "Ford F-150 Regular Cab 2007": [
        "Robust and functional truck design",
        "Balanced engine for utility and transport",
        "Engineered for towing and cargo tasks",
        "Full-size regular cab truck",
        "A classic model in Ford's F-series trucks"
    ],
    "Ford Focus Sedan 2007": [
        "Compact and modern sedan design",
        "Efficient engine for daily commuting",
        "Ideal for city and suburban driving",
        "Small economy sedan",
        "Represents Ford's commitment to compact cars"
    ],
    "Ford E-Series Wagon Van 2012": [
        "Spacious and practical van design",
        "V8 engine for reliable performance",
        "Perfect for family or commercial use",
        "Large utility van",
        "A versatile model in Ford's lineup"
    ],
    "Ford Fiesta Sedan 2012": [
        "Stylish and compact sedan design",
        "Fuel-efficient engine for urban use",
        "Designed for city commuting",
        "Small economy sedan",
        "Represents Ford's focus on affordability"
    ],
    "GMC Terrain SUV 2012": [
        "Modern and rugged SUV design",
        "Efficient engine for balanced performance",
        "Ideal for family trips and suburban driving",
        "Midsize luxury SUV",
        "A cornerstone in GMC's SUV offerings"
    ],
    "GMC Savana Van 2012": [
        "Spacious and robust van design",
        "Powerful engine for cargo and transport",
        "Built for commercial and utility tasks",
        "Large cargo van",
        "Represents GMC's focus on practicality"
    ],
    "GMC Yukon Hybrid SUV 2012": [
        "Eco-friendly and spacious SUV design",
        "Hybrid engine for efficiency and power",
        "Perfect for family and long-distance travel",
        "Large hybrid SUV",
        "Combines GMC's luxury with green technology"
    ],
    "GMC Acadia SUV 2012": [
        "Modern and versatile SUV design",
        "Smooth engine for comfortable driving",
        "Ideal for family and daily commuting",
        "Large luxury SUV",
        "Represents GMC's commitment to families"
    ],
    "GMC Canyon Extended Cab 2012": [
        "Compact and functional truck design",
        "Efficient engine for light-duty tasks",
        "Engineered for utility and transport",
        "Midsize extended cab truck",
        "A reliable model in GMC's lineup"
    ],
    "Geo Metro Convertible 1993": [
        "Compact and retro convertible design",
        "Fuel-efficient engine for urban driving",
        "Ideal for daily city commuting",
        "Small economy convertible",
        "Represents Geo's focus on affordability"
    ],
    "HUMMER H3T Crew Cab 2010": [
        "Bold and rugged truck design",
        "V8 engine delivering off-road capability",
        "Engineered for utility and adventure",
        "Midsize crew cab truck",
        "Represents HUMMER's adventurous spirit"
    ],
    "HUMMER H2 SUT Crew Cab 2009": [
        "Aggressive and commanding truck design",
        "Powerful engine for heavy-duty tasks",
        "Designed for off-road and rugged terrain",
        "Large crew cab truck",
        "A standout model in HUMMER's lineup"
    ],
    "Honda Odyssey Minivan 2012": [
        "Modern and spacious minivan design",
        "V6 engine delivering smooth performance",
        "Perfect for family trips and daily transport",
        "Large luxury minivan",
        "A cornerstone of Honda's family vehicle offerings"
    ],
    "Honda Odyssey Minivan 2007": [
        "Classic and practical minivan design",
        "Reliable engine for everyday use",
        "Engineered for family transport and cargo",
        "Large family minivan",
        "Represents Honda's commitment to practicality"
    ],
    "Honda Accord Coupe 2012": [
        "Sleek and sporty coupe design",
        "Efficient engine for spirited driving",
        "Ideal for highway and suburban commuting",
        "Compact luxury coupe",
        "A versatile model in Honda's lineup"
    ],
    "Honda Accord Sedan 2012": [
        "Elegant and modern sedan design",
        "Balanced engine for smooth performance",
        "Perfect for urban and highway driving",
        "Midsize luxury sedan",
        "Represents Honda's flagship sedan offering"
    ],
    "Hyundai Veloster Hatchback 2012": [
        "Futuristic and sporty hatchback design",
        "Turbocharged engine for zippy performance",
        "Engineered for city and suburban use",
        "Compact luxury hatchback",
        "Represents Hyundai's innovative design approach"
    ],
    "Hyundai Santa Fe SUV 2012": [
        "Modern and robust SUV design",
        "Smooth engine delivering balanced performance",
        "Perfect for family trips and cargo transport",
        "Midsize luxury SUV",
        "A dependable model in Hyundai's lineup"
    ],
    "Hyundai Tucson SUV 2012": [
        "Stylish and compact SUV design",
        "Efficient engine for smooth handling",
        "Ideal for urban and suburban driving",
        "Compact luxury SUV",
        "Represents Hyundai's focus on versatility"
    ],
    "Hyundai Veracruz SUV 2012": [
        "Elegant and spacious SUV design",
        "V6 engine delivering comfortable performance",
        "Perfect for family and long trips",
        "Large luxury SUV",
        "Combines Hyundai's practicality with luxury"
    ],
    "Hyundai Sonata Hybrid Sedan 2012": [
        "Eco-friendly and modern sedan design",
        "Hybrid engine offering efficiency and power",
        "Designed for city and highway commuting",
        "Midsize hybrid sedan",
        "Represents Hyundai's green technology efforts"
    ],
    "Hyundai Elantra Sedan 2007": [
        "Compact and reliable sedan design",
        "Efficient engine for smooth performance",
        "Perfect for daily commuting and errands",
        "Small economy sedan",
        "A popular model in Hyundai's compact lineup"
    ],
    "Hyundai Accent Sedan 2012": [
        "Compact and modern sedan design",
        "Fuel-efficient engine for daily commuting",
        "Ideal for city and suburban driving",
        "Small economy sedan",
        "Represents Hyundai's commitment to affordable vehicles"
    ],
    "Hyundai Genesis Sedan 2012": [
        "Luxurious and refined sedan design",
        "Powerful V6 engine with smooth performance",
        "Perfect for executive commuting and long trips",
        "Midsize luxury sedan",
        "A flagship model in Hyundai's luxury lineup"
    ],
    "Hyundai Sonata Sedan 2012": [
        "Stylish and spacious sedan design",
        "Balanced engine delivering reliable performance",
        "Ideal for family and daily commuting",
        "Midsize economy sedan",
        "Represents Hyundai's focus on modern design"
    ],
    "Hyundai Elantra Touring Hatchback 2012": [
        "Practical and compact hatchback design",
        "Efficient engine for versatile performance",
        "Perfect for family and cargo transport",
        "Compact economy hatchback",
        "A reliable model in Hyundai's hatchback offerings"
    ],
    "Hyundai Azera Sedan 2012": [
        "Elegant and modern sedan design",
        "V6 engine offering smooth and powerful performance",
        "Ideal for city and highway commuting",
        "Large luxury sedan",
        "Represents Hyundai's premium sedan lineup"
    ],
    "Infiniti G Coupe IPL 2012": [
        "Sporty and luxurious coupe design",
        "High-performance engine with agile handling",
        "Engineered for spirited highway driving",
        "Compact luxury coupe",
        "A flagship model in Infiniti's performance line"
    ],
    "Infiniti QX56 SUV 2011": [
        "Bold and commanding SUV design",
        "V8 engine with smooth performance",
        "Perfect for family trips and off-road use",
        "Large luxury SUV",
        "Represents Infiniti's premium SUV offerings"
    ],
    "Isuzu Ascender SUV 2008": [
        "Robust and practical SUV design",
        "Reliable engine for everyday performance",
        "Ideal for family transport and cargo needs",
        "Midsize economy SUV",
        "Represents Isuzu's focus on functionality"
    ],
    "Jaguar XK XKR 2012": [
        "Elegant and powerful coupe design",
        "Supercharged V8 engine for thrilling performance",
        "Engineered for high-speed luxury driving",
        "Large luxury sports coupe",
        "Represents Jaguar's commitment to performance"
    ],
    "Jeep Patriot SUV 2012": [
        "Rugged and functional SUV design",
        "Efficient engine for balanced performance",
        "Perfect for off-road and urban driving",
        "Compact economy SUV",
        "A versatile model in Jeep's lineup"
    ],
    "Jeep Wrangler SUV 2012": [
        "Iconic and rugged off-road design",
        "Powerful engine for challenging terrains",
        "Engineered for outdoor adventures",
        "Compact off-road SUV",
        "Represents Jeep's legendary off-road heritage"
    ],
    "Jeep Liberty SUV 2012": [
        "Modern and compact SUV design",
        "Efficient engine for reliable performance",
        "Ideal for family and cargo transport",
        "Compact luxury SUV",
        "A key model in Jeep's lineup"
    ],
    "Jeep Grand Cherokee SUV 2012": [
        "Bold and luxurious SUV design",
        "V6 engine delivering smooth performance",
        "Perfect for family and off-road trips",
        "Large luxury SUV",
        "Represents Jeep's flagship SUV offering"
    ],
    "Jeep Compass SUV 2012": [
        "Stylish and compact SUV design",
        "Efficient engine for balanced performance",
        "Ideal for urban and suburban use",
        "Compact economy SUV",
        "Represents Jeep's commitment to versatility"
    ],
    "Lamborghini Reventon Coupe 2008": [
        "Exotic and futuristic coupe design",
        "V12 engine delivering extreme performance",
        "Built for track and high-speed driving",
        "Compact hypercar coupe",
        "A limited-edition masterpiece by Lamborghini"
    ],
    "Lamborghini Aventador Coupe 2012": [
        "Aggressive and aerodynamic supercar design",
        "V12 engine for record-breaking performance",
        "Perfect for high-speed highway driving",
        "Large luxury sports coupe",
        "Represents Lamborghini's cutting-edge engineering"
    ],
    "Lamborghini Gallardo LP 570-4 Superleggera 2012": [
        "Lightweight and aerodynamic coupe design",
        "V10 engine for thrilling performance",
        "Engineered for track enthusiasts",
        "Compact hypercar coupe",
        "A benchmark in Lamborghini's sports car lineup"
    ],
    "Lamborghini Diablo Coupe 2001": [
        "Iconic and aggressive coupe design",
        "V12 engine delivering high-speed thrills",
        "Perfect for track and spirited highway driving",
        "Large luxury sports coupe",
        "A symbol of Lamborghini's '90s innovation"
    ],
    "Land Rover Range Rover SUV 2012": [
        "Luxurious and commanding SUV design",
        "V8 engine offering balanced off-road capability",
        "Ideal for family trips and outdoor adventures",
        "Large luxury SUV",
        "A flagship model in Land Rover's lineup"
    ],
    "Land Rover LR2 SUV 2012": [
        "Compact and versatile SUV design",
        "Efficient engine delivering smooth performance",
        "Perfect for urban and suburban use",
        "Compact luxury SUV",
        "Represents Land Rover's focus on accessibility"
    ],
    "Lincoln Town Car Sedan 2011": [
        "Elegant and traditional sedan design",
        "V8 engine for smooth and reliable performance",
        "Ideal for executive-level commuting",
        "Full-size luxury sedan",
        "Represents Lincoln's heritage in premium sedans"
    ],
    "MINI Cooper Roadster Convertible 2012": [
        "Charming and compact convertible design",
        "Efficient engine for zippy performance",
        "Perfect for city and open-top driving",
        "Small luxury convertible",
        "Represents MINI's playful and iconic style"
    ],
    "Maybach Landaulet Convertible 2012": [
        "Exquisite and ultra-luxury convertible design",
        "V12 engine delivering refined performance",
        "Designed for unparalleled luxury and comfort",
        "Large luxury convertible",
        "A hallmark of Maybach's engineering excellence"
    ],
    "Mazda Tribute SUV 2011": [
        "Practical and compact SUV design",
        "Efficient engine delivering smooth performance",
        "Ideal for urban and suburban use",
        "Compact economy SUV",
        "A dependable model in Mazda's lineup"
    ],
    "McLaren MP4-12C Coupe 2012": [
        "Futuristic and aerodynamic coupe design",
        "Twin-turbo V8 engine for thrilling performance",
        "Engineered for track enthusiasts and high-speed driving",
        "Compact hypercar coupe",
        "A symbol of McLaren's precision engineering"
    ],
    "Mercedes-Benz 300-Class Convertible 1993": [
        "Classic and elegant convertible design",
        "Smooth and reliable engine performance",
        "Ideal for luxurious open-top driving",
        "Midsize luxury convertible",
        "Represents Mercedes-Benz's timeless elegance"
    ],
    "Mercedes-Benz C-Class Sedan 2012": [
        "Modern and refined sedan design",
        "Turbocharged engine for balanced performance",
        "Perfect for city commuting and highway driving",
        "Compact luxury sedan",
        "A key model in Mercedes-Benz's executive lineup"
    ],
    "Mercedes-Benz SL-Class Coupe 2009": [
        "Sleek and aerodynamic coupe design",
        "V8 engine delivering smooth and powerful performance",
        "Ideal for spirited luxury driving",
        "Large luxury coupe",
        "Represents Mercedes-Benz's sports car excellence"
    ],
    "Mercedes-Benz E-Class Sedan 2012": [
        "Sophisticated and spacious sedan design",
        "Efficient engine delivering balanced performance",
        "Engineered for executive-level commuting",
        "Midsize luxury sedan",
        "A cornerstone of Mercedes-Benz's lineup"
    ],
    "Mercedes-Benz S-Class Sedan 2012": [
        "Luxurious and commanding sedan design",
        "V8 engine delivering exceptional performance",
        "Perfect for high-end commuting and long trips",
        "Full-size luxury sedan",
        "A flagship model in Mercedes-Benz's offerings"
    ],
    "Mercedes-Benz Sprinter Van 2012": [
        "Spacious and practical van design",
        "Powerful engine for utility and cargo use",
        "Ideal for commercial transport",
        "Large utility van",
        "Represents Mercedes-Benz's versatility and functionality"
    ],
    "Mitsubishi Lancer Sedan 2012": [
        "Sporty and modern sedan design",
        "Turbocharged engine for spirited performance",
        "Ideal for city and suburban driving",
        "Compact economy sedan",
        "A key model in Mitsubishi's lineup"
    ],
    "Nissan Leaf Hatchback 2012": [
        "Eco-friendly and compact hatchback design",
        "Electric motor for efficient and quiet performance",
        "Perfect for urban commuting",
        "Small electric hatchback",
        "Represents Nissan's innovation in green technology"
    ],
    "Nissan NV Passenger Van 2012": [
        "Spacious and functional van design",
        "V8 engine for reliable and smooth performance",
        "Ideal for commercial and passenger transport",
        "Large passenger van",
        "Represents Nissan's focus on utility vehicles"
    ],
    "Nissan Juke Hatchback 2012": [
        "Unique and sporty hatchback design",
        "Turbocharged engine for agile performance",
        "Engineered for urban and suburban use",
        "Compact luxury hatchback",
        "Represents Nissan's innovative styling"
    ],
    "Nissan 240SX Coupe 1998": [
        "Classic and sporty coupe design",
        "Efficient engine with smooth performance",
        "Ideal for track enthusiasts and daily driving",
        "Compact sports coupe",
        "A popular model in Nissan's lineup"
    ],
    "Plymouth Neon Coupe 1999": [
        "Simple and practical coupe design",
        "Fuel-efficient engine for urban commuting",
        "Perfect for city driving",
        "Compact economy coupe",
        "A reliable model in Plymouth's lineup"
    ],
    "Porsche Panamera Sedan 2012": [
        "Luxurious and aerodynamic sedan design",
        "Powerful engine for high-speed performance",
        "Built for executive-level commuting",
        "Large luxury sports sedan",
        "Represents Porsche's blend of luxury and performance"
    ],
    "Ram C/V Cargo Van Minivan 2012": [
        "Functional and spacious cargo van design",
        "Efficient engine for utility tasks",
        "Ideal for commercial transport",
        "Large utility van",
        "Represents Ram's dedication to practical vehicles"
    ],
    "Rolls-Royce Phantom Drophead Coupe Convertible 2012": [
        "Exquisite and luxurious convertible design",
        "V12 engine delivering smooth performance",
        "Engineered for unparalleled open-top luxury",
        "Large luxury convertible",
        "A masterpiece in Rolls-Royce's lineup"
    ],
    "Rolls-Royce Ghost Sedan 2012": [
        "Elegant and refined sedan design",
        "V12 engine delivering seamless performance",
        "Perfect for executive-level commuting",
        "Full-size luxury sedan",
        "Represents Rolls-Royce's commitment to luxury"
    ],
    "Rolls-Royce Phantom Sedan 2012": [
        "Timeless and commanding sedan design",
        "V12 engine delivering exceptional performance",
        "Ideal for high-end luxury commuting",
        "Full-size luxury sedan",
        "A flagship model in Rolls-Royce's lineup"
    ],
    "Scion xD Hatchback 2012": [
        "Compact and modern hatchback design",
        "Efficient engine for urban commuting",
        "Ideal for city and suburban driving",
        "Small economy hatchback",
        "Represents Scion's youthful and practical styling"
    ],
    "Spyker C8 Convertible 2009": [
        "Exotic and luxurious convertible design",
        "High-performance V8 engine",
        "Built for spirited open-top driving",
        "Compact luxury convertible",
        "Represents Spyker's craftsmanship and exclusivity"
    ],
    "Spyker C8 Coupe 2009": [
        "Sleek and aerodynamic coupe design",
        "V8 engine delivering exceptional performance",
        "Engineered for track and high-speed driving",
        "Compact luxury coupe",
        "A symbol of Spyker's precision engineering"
    ],
    "Suzuki Aerio Sedan 2007": [
        "Simple and practical sedan design",
        "Efficient engine for reliable performance",
        "Ideal for city and suburban commuting",
        "Small economy sedan",
        "Represents Suzuki's focus on affordable cars"
    ],
    "Suzuki Kizashi Sedan 2012": [
        "Modern and sporty sedan design",
        "Balanced engine for smooth performance",
        "Perfect for daily commuting",
        "Compact luxury sedan",
        "Represents Suzuki's refinement in sedan offerings"
    ],
    "Suzuki SX4 Hatchback 2012": [
        "Compact and practical hatchback design",
        "Efficient engine for versatile driving",
        "Ideal for city and suburban transport",
        "Small economy hatchback",
        "Represents Suzuki's commitment to functionality"
    ],
    "Suzuki SX4 Sedan 2012": [
        "Stylish and modern sedan design",
        "Efficient engine for reliable performance",
        "Perfect for urban and suburban commuting",
        "Small economy sedan",
        "A practical model in Suzuki's lineup"
    ],
    "Tesla Model S Sedan 2012": [
        "Futuristic and sleek sedan design",
        "Electric motor for high performance",
        "Perfect for long-distance travel",
        "Full-size luxury electric sedan",
        "Represents Tesla's innovation in electric vehicles"
    ],
    "Toyota Sequoia SUV 2012": [
        "Spacious and commanding SUV design",
        "V8 engine for balanced performance",
        "Ideal for family and off-road adventures",
        "Large luxury SUV",
        "A reliable model in Toyota's SUV lineup"
    ],
    "Toyota Camry Sedan 2012": [
        "Modern and practical sedan design",
        "Efficient engine for daily commuting",
        "Perfect for urban and suburban use",
        "Midsize economy sedan",
        "A cornerstone in Toyota's sedan offerings"
    ],
    "Toyota Corolla Sedan 2012": [
        "Compact and reliable sedan design",
        "Fuel-efficient engine for city driving",
        "Ideal for daily commuting",
        "Small economy sedan",
        "Represents Toyota's focus on affordability"
    ],
    "Toyota 4Runner SUV 2012": [
        "Rugged and versatile SUV design",
        "V6 engine for off-road performance",
        "Engineered for family and adventure trips",
        "Midsize economy SUV",
        "A dependable model in Toyota's SUV lineup"
    ],
    "Volkswagen Golf Hatchback 2012": [
        "Modern and sporty hatchback design",
        "Efficient engine for smooth performance",
        "Ideal for urban and suburban driving",
        "Compact luxury hatchback",
        "A key model in Volkswagen's lineup"
    ],
    "Volkswagen Golf Hatchback 1991": [
        "Classic and practical hatchback design",
        "Reliable engine for everyday use",
        "Perfect for urban and suburban commuting",
        "Small economy hatchback",
        "Represents Volkswagen's focus on practicality"
    ],
    "Volkswagen Beetle Hatchback 2012": [
        "Iconic and retro-modern hatchback design",
        "Efficient engine delivering balanced performance",
        "Ideal for city and suburban use",
        "Compact economy hatchback",
        "Represents Volkswagen's legacy of innovation"
    ],
    "Volvo C30 Hatchback 2012": [
        "Stylish and compact hatchback design",
        "Balanced engine delivering smooth performance",
        "Perfect for urban commuting",
        "Small luxury hatchback",
        "Represents Volvo's focus on modern design"
    ],
    "Volvo 240 Sedan 1993": [
        "Classic and durable sedan design",
        "Reliable engine for everyday use",
        "Engineered for city and suburban driving",
        "Midsize economy sedan",
        "A legacy model in Volvo's lineup"
    ],
    "Volvo XC90 SUV 2007": [
        "Spacious and versatile SUV design",
        "V6 engine delivering reliable performance",
        "Ideal for family and long-distance travel",
        "Large luxury SUV",
        "A flagship model in Volvo's SUV offerings"
    ],
    "smart fortwo Convertible 2012": [
        "Compact and efficient convertible design",
        "Electric motor for urban commuting",
        "Ideal for city and short-distance driving",
        "Small electric convertible",
        "Represents Smart's innovation in urban mobility"
    ]
}